# LightFM runbook

## Loading libraries and setting parameters

In [45]:
import warnings
warnings.filterwarnings('ignore')

import sys

sys.path.append("../../..")

from src.common import helpers, config
from src.models.lightfm import LFM
from src.evaluation.evaluator import Evaluator

In [46]:
MODEL_NAME = "lightfm"

paths = config.Paths(
    dataset_name="jobs_published-part",
    target_users_name="all",
    model_name=MODEL_NAME,
)

In [49]:
no_components=30
k=5
n=3
learning_schedule="adagrad"
loss="warp-kos"
learning_rate=0.05
rho=0.95
epsilon=1e-06
item_alpha=0.0
user_alpha=0.0
max_sampled=10
random_state=42
epochs=10

N_RECOMMENDATIONS = 10

## Model training

In [50]:
model = LFM(
    no_components=no_components,
    k=k,
    n=n,
    learning_schedule=learning_schedule,
    loss=loss,
    learning_rate=learning_rate,
    rho=rho,
    epsilon=epsilon,
    item_alpha=item_alpha,
    user_alpha=user_alpha,
    max_sampled=max_sampled,
    random_state=random_state,
    epochs=epochs,
)

model.load_interactions(paths.train_and_validation)

model.preprocess()

model.fit()

Epoch: 100%|████████████████████████████████████| 10/10 [00:07<00:00,  1.42it/s]


## Generating recommendations

In [51]:
target_users = model.load_target_users(paths.target_users)

In [52]:
recommendations = model.recommend(
    target_users=target_users, n_recommendations=N_RECOMMENDATIONS, show_progress=True
)

model.save_recommendations(recommendations, paths.recommendations)

20738it [00:13, 1563.33it/s]


In [53]:
evaluator = Evaluator(
    recommendations_path=paths.recommendations_folder,
    test_path=paths.test,
    k=10,
    models_to_evaluate=[MODEL_NAME],
)

evaluator.prepare()

evaluator.evaluate_models()

20738it [00:01, 15717.46it/s]


In [54]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
lightfm,0.009056,0.043029,0.013826,0.010429,0.02773,0.017626,0.032799,0.52091,0.081686,0.986821,0.519567,6.902122,0.890953,0.0,0.0


In [35]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
lightfm,0.002131,0.010143,0.003244,0.002449,0.006102,0.003551,0.007477,0.504458,0.020494,1.0,0.001969,2.369485,0.998878,0.0,0.0
